# You suck at shell scripting!

You do! You're aweful! You are as bad at shell scripting
as Helen Zille is at Twitter. However clever that 
140 character bash one liner seemed at the time,
you know that you regretted it once it became an
integral part of your production build pipeline. This
talk will show you how to write maintainable
Command Line Interfaces with `click` and `pathlib`
that won't leave you trying to rewrite your git commit
history after the next production server outage.


In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/U_X5uR7VC4M" frameborder="0" allowfullscreen></iframe>


## Aim

  * demonstrate
      * pathlib
      * click
  * worked example for file deduplication

## shell scripts

  * Provide some examples of badly written shell scripts
  * Why I've resisted using Python for shell scripting.
      * os.path

## pathlib

Path handling the right way

In [ ]:
from pathlib import Path
p = Path('.')
p

In [ ]:
print(str(p))
print(p.absolute())

In [ ]:
p = p.absolute()
print(p.as_posix())

In [ ]:
print(p.as_uri())

In [ ]:
p.is_dir()

In [ ]:
p.is_file()

In [ ]:
p.parent

In [ ]:
p.relative_to(p.parent)

In [ ]:
# subdirectories
[x for x in p.iterdir() if x.is_dir()]

In [ ]:
# files
[x for x in p.iterdir() if x.is_file()]

In [ ]:
list(p.rglob('*.ipynb'))

In [ ]:
p = Path('.') / 'newdir'
p

In [ ]:
p.exists()

In [ ]:
p.mkdir()

In [ ]:
p.exists()

In [ ]:
q = p / 'newfile.txt'
q

In [ ]:
with q.open('wt') as f:
    f.write('The quick brown fox jumped over the lazy dog.')

In [ ]:
q.exists() and q.is_file()

In [ ]:
q.read_text()

In [ ]:
q.unlink()

In [ ]:
q.exists()

In [ ]:
p.rmdir()

In [ ]:
p.exists()

# CLIs

  * 40 years of bad branding as **Command Line Intefaces**
  * Who knows what that is?
  * Rebrand as _"Chatbot Like Interface"_

## Click 

  * **"Command Line Interface Creation Kit"** or 
  * **"Chatbot Like Interface Creation Kit"**?

Beautiful CLI APIs
  * automatic help page generation
  * parameter validation
  * arbitrary nesting of commands
  * supports lazy loading of subcommands at runtime

In [ ]:
%%HTML
<iframe width="1024" height="400" src="http://click.pocoo.org/5/"></iframe>

### To cover
  
* arguments vs options
* parameter types

### Basic example

```python
# %load hello.py
import click

@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name', help='The person to greet.')
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo('Hello {}!'.format(name))

if __name__=='__main__':
    hello()
```

In [37]:
!python hello.py --name Tobias --count 3

Hello Tobias!
Hello Tobias!
Hello Tobias!


## Extended example

In [38]:
# %load ysass.py

In [54]:
Path('file1.txt').write_text('Hello world!')
Path('file2.txt').write_text('Hello world!')

12

In [60]:
!python ysass.py

Usage: ysass.py [OPTIONS] COMMAND1 [ARGS]... [COMMAND2 [ARGS]...]...

Options:
  --help  Show this message and exit.

Commands:
  digests  Generate the digests for all files in a path
  show


In [72]:
!python ysass.py digests show

e49c605a34d57cf04b7e528f0181f2b4 .gitignore
86fb269d190d2c85f6e0468ceca42a20 file1.txt
86fb269d190d2c85f6e0468ceca42a20 file2.txt
3d7b352884196e284b5e8a2707be7ef3 hello.py
0f311c476d22dcb4b0085675a79d717b SAFA-2017.ipynb
77301da323346c17475a74d0d2bf7d47 You suck at shell scripting.ipynb
1296fe6cf607e831f9246dd9c174640d ysass.py


In [67]:
!python ysass.py digests --help

Usage: ysass.py digests [OPTIONS]

  Generate the digests for all files in a path

Options:
  -h, --hash-type [md5|sha1]
  -p, --path DIRECTORY
  --help                      Show this message and exit.


In [69]:
!python ysass.py digests -p file1.txt show

Usage: ysass.py digests [OPTIONS]

Error: Invalid value for "-p" / "--path": Directory "file1.txt" is a file.


In [68]:
!python ysass.py digests -h sha1 show

3b78bdedca59dc7ab66176f235b8b56fc9888d0f .gitignore
d3486ae9136e7856bc42212385ea797094475802 file1.txt
d3486ae9136e7856bc42212385ea797094475802 file2.txt
69c289606a5b6159c490458dcd4139dd5f0faf8b hello.py
5fd2b64ca38b06f02a0203aca74d4a6bb08f8648 SAFA-2017.ipynb
aa356c56cf28d8e265f165e76497f1e4e9f08549 You suck at shell scripting.ipynb
a838d6e8e2c6f918c4071abe115be68d44606724 ysass.py


In [70]:
!python ysass.py digests -h sha256 show

Usage: ysass.py digests [OPTIONS]

Error: Invalid value for "--hash-type" / "-h": invalid choice: sha256. (choose from md5, sha1)


## setuptools integration